In [1]:
import numpy as np
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt 
import matplotlib.style as stl 
from tensorflow import keras 
from keras.preprocessing.text import Tokenizer
from nltk.corpus import stopwords
import string
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Dense, LSTM, Embedding,Dropout,SpatialDropout1D,Conv1D,MaxPooling1D,GRU,BatchNormalization
import warnings
warnings.filterwarnings('ignore')
stl.use('ggplot')
from tqdm import tqdm
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer
import regex as re
import time
import pickle

In [2]:
order=pd.read_csv("../input/categories/order.csv")

In [3]:
order.head()

,document_id,status,document_url,media_type,media_link,relative_media_path,is_media_downloaded,transcription_text,transcription_text_relative_path,from_time,to_time,project_id,Category,Unnamed: 13,Unnamed: 14
0,NMAAHC-004567391_00220,Complete,https://transcription.si.edu/transcribe/9649/N...,image,https://ids.si.edu/ids/deliveryService?max_w=&...,/9649/image/NMAAHC-004567391_00220.jpg,1,[B 126 ENCL]\r\nR & G. R.R.\r\nTHIS ORDER NOT ...,/9649/transcriptions/NMAAHC-004567391_00220.txt,\N,\N,9649,Orders/Circulars,NaN,NaN
1,NMAAHC-004567391_00221,Complete,https://transcription.si.edu/transcribe/9649/N...,image,https://ids.si.edu/ids/deliveryService?max_w=&...,/9649/image/NMAAHC-004567391_00221.jpg,1,"Certified Copy of Order, or Extract, on which ...",/9649/transcriptions/NMAAHC-004567391_00221.txt,\N,\N,9649,Orders/Circulars,NaN,NaN
2,NMAAHC-004567391_00873,Complete,https://transcription.si.edu/transcribe/9652/N...,image,https://ids.si.edu/ids/deliveryService?max_w=&...,/9652/image/NMAAHC-004567391_00873.jpg,1,"Bureau of Refugees, Freedmen and Abandoned Lan...",/9652/transcriptions/NMAAHC-004567391_00873.txt,\N,\N,9652,Orders/Circulars,NaN,NaN
3,NMAAHC-004567392_00226,Complete,https://transcription.si.edu/transcribe/9780/N...,image,https://ids.si.edu/ids/deliveryService?max_w=&...,/9780/image/NMAAHC-004567392_00226.jpg,1,[L26]\r\n\r\nHEADQUARTERS DEPARTMENT OF NORTH ...,/9780/transcriptions/NMAAHC-004567392_00226.txt,\N,\N,9780,Orders/Circulars,NaN,NaN
4,NMAAHC-004567392_00227,Complete,https://transcription.si.edu/transcribe/9780/N...,image,https://ids.si.edu/ids/deliveryService?max_w=&...,/9780/image/NMAAHC-004567392_00227.jpg,1,IV. Animals will be sold in the counties wher...,/9780/transcriptions/NMAAHC-004567392_00227.txt,\N,\N,9780,Orders/Circulars,NaN,NaN


In [4]:
order.shape

(7668, 15)

In [5]:
order_text=order["transcription_text"]

In [6]:
print(order_text[0])

[B 126 ENCL]
R & G. R.R.
THIS ORDER NOT TRANSFERABLE.
U.S. ORDER FOR TRANSPORTATION 
NOT VALID UNTIL COUNTERSIGNED BY THE OFFICER ISSUING.

C. NO. 67487

Raleigh N.C. Aug 6 1866
Transport Samuel Gale
[[strikethrough]] Co. Regiment [[/strikethrough]] Freedman
From Warrenton N.C. to Raleigh N.C.
En Route from Warrenton N.C. to Raleigh N.C.

Signature of Officer issuing the order
E. Cranford
2 Lt 37th U.S.C. Inft AAQM

By order of the Quarter Master Gen'l. 
Lewis B. Parsons
Brig. Gen. & Chief of Rail & River Transptn.

[[image]] The holder of this order, or the officer in charge of troops, is required to fill the receipt below before signing, stating the exact number of men transported and the places from and to which transportation has been furnished.

Aug 1866
I CERTIFY, on honor, that Ral. & Gaston R.R. has furnished transportation for Myself From Warrenton N.C. to Raleigh N.C. in compliance with the above order.

Major & Knapp, 110 Broadway, New York. 



In [7]:
print(order_text[10])

Bureau of Refugees, Freedmen & Ab'nd. Lands,
Office Sub Assistant Commissioner,
Greensboro, N.C., August 9, 1867.

Circular No. 3.

I would respectfully inform the colored people of this Sub District, that in compliance with Circular Order, No. 14, C.S. Hd. Qr's. Asst. Com'r, State of N.C., I will keep a record of applications for labor at this office for the purpose of equalizing the supply, as recommended in the above order, and obtain the highest possible wages for applicants.
All in want of employment are invited to register their names, strength of families, residence, occupation and price of labor demanded. I also invite all who desire to employ, or hire hands, to register their wants, stating in full, the terms on which they desire to make labor contracts.
A.W. Bolenius,
Capt. V.R.C and Bre'vt Maj. U.S. Vols.
Sub Asst. Com'r.
Official:


In [8]:
print(order_text[100])

Genl order no 156

A, G, O,

War Department.
Adjutant Generals Office
Washington DC.
Octbr 26. 1865.

General Orders}
No 156.}




In [9]:
print(order_text[5000])

Bureau of Refugees, Freedmen & Abandoned Lands,
Headquarters Assistant Commissioner State of North Carolina,
Raleigh, N.C., Septem 16" 1867.

Circular Letter
The Asst Commissioner directs that you forward to this office, a report of the operations of the Bureau in your district for the current year, on or before Sept. 25" inst., In the cases where districts have been recently established, the report will cover the period of its organization. Statistics on all points, observations and suggestions, will be given in detail.

Very Resp'y,
Your Obed Serv't
Hugo Hillebrandt
Capt. V.R.C. & A.A.A.G

Bvt Maj. H D Norton 
Sub Asst Comr


In [10]:
print(order_text[7000])

35

Cleveland: and will be obeyed and respected accordingly.
He will at once enter upon the discharge of his duties.

By order of,
Bvt Major General Clinton B. Fisk:
Asst: Commissioner
for Ky. and Tenn.
J.E. Jacobs.
Bv. Lieut. Col. and A. A. G.

36

Bureau Refugees, Freedmen and Aban'd Lands,
Assistant Commissioner's Office
Nashville, Tenn. January 27th 1866.

Special Orders
No. 17.

[[left margin]] 
John Overton
Nashville,
Tenn.
Capt. Staley,
Memphis,
Tenn.
B.B. Waddell
Memphis,
Tenn.
x
Major Wills A. Q. M.
Nashville, Tenn.
Maj Gen'l O.O. Howard. 
[[/left margin]]

#. 1. In obedience to Orders from the President of the United States, all control over any and all real property within the limits of this District formerly owned by John Overton and held by this Bureau as abandoned, is, hereby relinquished on the following conditions,
1st Lessees remain in possession until the expiration of their Leases.
2nd This Order shall not be construed as giving any claim upon the Government for dama

In [11]:
stop_words = set(stopwords.words('english'))
punctuation = string.punctuation
stemmer = SnowballStemmer(language="english")
lemmatizer = WordNetLemmatizer()
stop_words.remove("of")
punct='!"#$%&\'()*+-/<=>?@[\\]^_`{|}~'
def process(s):
    s= re.sub("([\[]).*?([\]])"," ",s)
    for p in punct:
        s = s.replace(p, '')
    s = word_tokenize(s)
    s = [w for w in s if not w in stop_words] #optional
    return s
for i in tqdm(range(len(order_text))):
    order_text[i] = process(order_text[i])
for i in tqdm(range(len(order_text))):
    order_text[i] = " ".join(order_text[i])

100%|██████████| 7668/7668 [00:03<00:00, 2519.12it/s]


In [12]:
order_text[7000]

'35 Cleveland : obeyed respected accordingly . He enter upon discharge of duties . By order of , Bvt Major General Clinton B. Fisk : Asst : Commissioner Ky. Tenn. J.E . Jacobs . Bv . Lieut . Col. A . A. G. 36 Bureau Refugees , Freedmen Aband Lands , Assistant Commissioners Office Nashville , Tenn. January 27th 1866 . Special Orders No . 17 . John Overton Nashville , Tenn. Capt . Staley , Memphis , Tenn. B.B . Waddell Memphis , Tenn. x Major Wills A. Q. M. Nashville , Tenn. Maj Genl O.O . Howard . . 1 . In obedience Orders President of United States , control real property within limits of District formerly owned John Overton held Bureau abandoned , , hereby relinquished following conditions , 1st Lessees remain possession expiration of Leases . 2nd This Order shall construed giving claim upon Government damages , rents . By order of Brevet Major General Clinton B. Fisk : Asst : Comr Ky Tenn. J.E.Jacobs . Bt Lt. Col. A . A . General .'

In [13]:
!pip3 install Cython
!pip install transformers
!pip install flair

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.9/401.9 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.3/46.3 kB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
  Installing build dependencies ... - \ | / - \ done
  Getting requirements to build wheel ... - \ | / - done
  Preparing metadata (pyproject.toml) ... - \ | / - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 788.5/788.5 kB 6.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 12.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 36.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 6.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
  Created wheel for gdown: filename=gdown-4.4.0-p

In [14]:
from flair.models import SequenceTagger

tagger = SequenceTagger.load("flair/ner-english-ontonotes-large")

Downloading:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

2022-06-27 19:14:32,884 loading file /root/.flair/models/ner-english-ontonotes-large/2da6c2cdd76e59113033adf670340bfd820f0301ae2e39204d67ba2dc276cc28.ec1bdb304b6c66111532c3b1fc6e522460ae73f1901848a4d0362cdf9760edb1


Downloading:   0%|          | 0.00/616 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.83M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/8.68M [00:00<?, ?B/s]

2022-06-27 19:15:00,862 SequenceTagger predicts: Dictionary with 76 tags: <unk>, O, B-CARDINAL, E-CARDINAL, S-PERSON, S-CARDINAL, S-PRODUCT, B-PRODUCT, I-PRODUCT, E-PRODUCT, B-WORK_OF_ART, I-WORK_OF_ART, E-WORK_OF_ART, B-PERSON, E-PERSON, S-GPE, B-DATE, I-DATE, E-DATE, S-ORDINAL, S-LANGUAGE, I-PERSON, S-EVENT, S-DATE, B-QUANTITY, E-QUANTITY, S-TIME, B-TIME, I-TIME, E-TIME, B-GPE, E-GPE, S-ORG, I-GPE, S-NORP, B-FAC, I-FAC, E-FAC, B-NORP, E-NORP, S-PERCENT, B-ORG, E-ORG, B-LANGUAGE, E-LANGUAGE, I-CARDINAL, I-ORG, S-WORK_OF_ART, I-QUANTITY, B-MONEY


In [15]:
from flair.models import SequenceTagger

tagger1 = SequenceTagger.load("flair/ner-english")

Downloading:   0%|          | 0.00/432M [00:00<?, ?B/s]

2022-06-27 19:15:15,425 loading file /root/.flair/models/ner-english/4f4cdab26f24cb98b732b389e6cebc646c36f54cfd6e0b7d3b90b25656e4262f.8baa8ae8795f4df80b28e7f7b61d788ecbb057d1dc85aacb316f1bd02837a4a4
2022-06-27 19:15:17,356 SequenceTagger predicts: Dictionary with 20 tags: <unk>, O, S-ORG, S-MISC, B-PER, E-PER, S-LOC, B-ORG, E-ORG, I-PER, S-PER, B-MISC, I-MISC, E-MISC, I-ORG, B-LOC, E-LOC, I-LOC, <START>, <STOP>


In [16]:
from flair.models import SequenceTagger

tagger2 = SequenceTagger.load("flair/ner-english-large")

Downloading:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

2022-06-27 19:16:29,972 loading file /root/.flair/models/ner-english-large/07301f59bb8cb113803be316267f06ddf9243cdbba92a4c8067ef92442d2c574.554244d3476d97501a766a98078421817b14654496b86f2f7bd139dc502a4f29
2022-06-27 19:16:48,564 SequenceTagger predicts: Dictionary with 20 tags: <unk>, O, S-ORG, S-MISC, B-PER, E-PER, S-LOC, B-ORG, E-ORG, I-PER, S-PER, B-MISC, I-MISC, E-MISC, I-ORG, B-LOC, E-LOC, I-LOC, <START>, <STOP>


# ontonotes

In [17]:
lst=[]
label=[]
Per=[]
Date=[]
GPE=[]
ORG=[]
LOC=[]
Bulding=[]
Cardinal=[]
from flair.data import Sentence

order_onto=pd.DataFrame({"order_text":"", "PERSON-ontonotes":[], "GPE-ontonotes":"","ORG-ontonotes":"", "Date-ontonotes":""})
for i in range(7668):
        sentencess =Sentence(order_text[i])
        order_onto.loc[i,"order_text"]=order_text[i]
        tagger.predict(sentencess)
        dc={}
        P=[]
        D=[]
        G=[]
        L=[]
        O=[]
        B=[]
        C=[]
        for entity in sentencess.get_spans('ner'):
                     
                     if (entity.get_label("ner").score > 0.97) & (entity.get_label("ner").value != "LAW")& (entity.get_label("ner").value != "QUANTITY") & (entity.get_label("ner").value != "WORK_OF_ART") & (entity.get_label("ner").value != "LAW")& (entity.get_label("ner").value != "QUANTITY") & (entity.get_label("ner").value != "EVENT")  :
                        dc[entity.text]= entity.get_label("ner").value 
        
        
        for j in dc :
                if dc[j]=="PERSON":
                      P.append(j)
                elif dc[j]=="ORG":
                      O.append(j)
                elif dc[j]=="DATE":
                      D.append(j)
                elif dc[j]=="GPE":
                      G.append(j)
                elif dc[j]=="LOC":
                      L.append(j)
                elif dc[j]=="FAC":
                      B.append(j)
                elif dc[j]=="CARDINAL":
                      C.append(j)
        Per.append(P)
        Date.append(D)
        ORG.append(O)
        GPE.append(G)
        LOC.append(L)
        Bulding.append(B)
        Cardinal.append(C)
        
            
order_onto["Date-ontonotes"]=Date
order_onto["PERSON-ontonotes"]=Per
order_onto["GPE-ontonotes"]=GPE
order_onto["ORG-ontonotes"]=ORG 
order_onto["LOC-ontonotes"]=LOC 
order_onto["Building-ontonotes"]=Bulding
order_onto["Building-ontonotes"]=Cardinal

2022-06-27 19:17:00,318 Warning: An empty Sentence was created! Are there empty strings in your dataset?
2022-06-27 19:17:50,627 Warning: An empty Sentence was created! Are there empty strings in your dataset?
2022-06-27 19:24:01,521 Warning: An empty Sentence was created! Are there empty strings in your dataset?


In [18]:
order_onto["order_text"]=order_text

In [19]:
order_onto.head()

,order_text,PERSON-ontonotes,GPE-ontonotes,ORG-ontonotes,Date-ontonotes,LOC-ontonotes,Building-ontonotes
0,R G. R.R . THIS ORDER NOT TRANSFERABLE . U.S. ...,"[E . Cranford, Lewis B. Parsons, Major Knapp]","[Raleigh N.C, Warrenton N.C, New York]",[Samuel Gale Co .],"[Aug 6 1866, Aug 1866]",[],[67487]
1,"Certified Copy of Order , Extract , within tra...",[E. Crawford],[Raleigh N.C],[],[Aug 6 1866],[],[]
2,"Bureau of Refugees , Freedmen Abandoned Lands ...","[KELLEY, EDWARD McPHERSON, JONATHAN WORTH]","[Raleigh, N.C., United States, Alabama, North ...","[Bureau of Refugees, House of Representatives,...","[May 8th , 1866, May 1 , 1866, 1st]",[],"[2d, III]"
3,"HEADQUARTERS DEPARTMENT OF NORTH CAROLINA , Ra...",[Joseph E. Johnston],"[Raleigh, North Carolina]","[HEADQUARTERS DEPARTMENT OF NORTH CAROLINA, He...","[January 12th , 1866]",[],"[4, II, III]"
4,IV . Animals sold counties collected unless wo...,"[J A . Campbell, James H Anderson]",[U.S.],"[Headquarters, Government]","[first day of February , proximo]",[],[VI]


# Post processing for dates

In [20]:
order_onto["Date-ontonotes"]

0                    [Aug 6 1866, Aug 1866]
1                              [Aug 6 1866]
2       [May 8th , 1866, May 1 , 1866, 1st]
3                     [January 12th , 1866]
4         [first day of February , proximo]
                       ...                 
7663                                     []
7664                       [Nov. 28 , 1866]
7665                       [Nov. 28 , 1866]
7666                       [Nov. 28 , 1866]
7667                       [Nov. 28 , 1866]
Name: Date-ontonotes, Length: 7668, dtype: object

In [21]:
G=[]
for i in order_onto["Date-ontonotes"]:
    M=[]
    for j in i:
        date=re.search(r'^\d\.*.*\d\d$' ,j)
        date2=re.search(r'^[A-Z]\.*.*\d\.*.*\d\d$' ,j)
        M.append(date)
        M.append(date2)
    G.append(M)
    

GF=[]
for i in G:
    Final_date=[]
    for j in i:
        if j!=None:
            Final_date.append(j.group())
    GF.append(Final_date)
    
            
      

In [22]:
len(GF)

7668

In [23]:
order_onto["postDate-ontonotes"]=GF

In [24]:
G=[]
for i in order_onto["postDate-ontonotes"]:
    M=[]
    for j in i:
        Final_date1=re.sub(r'([0-9]{1,2})\.*.*\s([A-Z][a-z]\S*)\.*.*\s([0-9]{2,4})',r'\1-\2-\3' ,j)
        Final_date2=re.sub(r'([A-Z]\.*.*)\s([0-9]{1,2})\.*.*\s([0-9]{2,4})',r'\2-\1-\3' ,j)
        final_date3=re.sub(r'([A-Z][a-z]\S*)\.*.*\s([A-Z][a-z]\S*)\.*.*\s([0-9]{2,4})',r'\1-\2-\3' ,j)
        final_date4=re.sub(r'(^[A-Za-z]\S*.*.)day\.*.*\s([A-Z][a-z]\S*)\.*.*\s([0-9]{2,4})',r'\1-\2-\3' ,j)
        if Final_date1 != j:   
            M.append(Final_date1)
        if Final_date2 != j :
            M.append(Final_date2)
        if final_date3 != j :
            M.append(final_date3)
        if final_date4 != j :
            M.append(final_date4)
    G.append(M)
    

order_onto["postDate-ontonotes"]=G

# standardizing dates

In [25]:
NG=[]
for i in G:
    ng=[]
    for j in i:
        
        date=re.findall(r'([0-9]{1,2})-([A-Z][a-z]\S*)-([0-9]{2,4})',j)
        if date !=[]:
            dates=[]
            for k in date:
                for l in k:
                    dates.append(l)
                
                
                if dates[1] in ["August","Aug","augt","AUGUST","Augst","Augt","AUG","Augusta"]:
                    dates[1]= "August"
                elif dates[1] in ["September","Sept","SEP","Sep","Septr","SEPTEMBER",'Sepr','Septh']:
                    dates[1]= "Septembre"
                elif dates[1] in ["October","Oct","OCT","OCTOBER"]:
                    dates[1]= "October"
                elif dates[1] in ["November","NOV","Nov","Novmbr","Nvbr","Novb","NOVEMBER", "The"]:
                    dates[1]= "November"
                elif dates[1] in ["December","Dec","Decbr","DEC","DECEMBER","Decber"]:
                    dates[1]= "December"
                elif dates[1] in ["January","Janry","Jan","Jany","JAN","Janur","JANUARY"]:
                    dates[1]= "January"
                elif dates[1] in ["February","Feby","Febry","FEB","FEBRUARY"]:
                    dates[1]= "February"
                elif dates[1] in ["March","Mar"]:
                    dates[1]= "March"
                elif dates[1] in ["April","Apl","Aprl","APRIL","Apr"]:
                    dates[1]= "April"
                elif dates[1] in ["May","MAY"]:
                    dates[1]= "May"
                elif dates[1] in ["JUN","Jun","June"]:
                    dates[1]= "June"
                elif dates[1] in ["JULY","July","JUL"]:
                    dates[1]= "July"
                j = dates[0]+"-"+dates[1]+"-"+dates[2]
                ng.append(j)
        else :
                date = re.findall(r'(^[A-Za-z]\S*.*.)-([A-Z][a-z]\S*)-([0-9]{2,4})',j ) 
                dates=[]
                for k in date:
                    for l in k:
                        dates.append(l)
                
                
                    if dates[1] in ["August","Aug","augt","AUGUST","Augst",'Augt',"AUG","Augusta"]:
                        dates[1]= "August"
                    elif dates[1] in ["September","Sept","SEP","Sep","Septr","SEPTEMBER","Sepr","Septh", "This", "This "]:
                        dates[1]= "Septembre"
                    elif dates[1] in ["October","Oct","OCT","OCTOBER"]:
                        dates[1]= "October"
                    elif dates[1] in ["November","NOV","Nov","Novmbr","Nvbr","Novb","NOVEMBER"]:
                        dates[1]= "November"
                    elif dates[1] in ["December","Dec","Decbr","DEC","DECEMBER","Decber"]:
                        dates[1]= "December"
                    elif dates[1] in ["January","Janry","Jan","Jany","JAN","Janur","JANUARY"]:
                        dates[1]= "January"
                    elif dates[1] in ["February","Feby","Febry","FEB","FEBRUARY"]:
                        dates[1]= "February"
                    elif dates[1] in ["March","Mar"]:
                        dates[1]= "March"
                    elif dates[1] in ["April","Apl","Aprl","APRIL","Apr"]:
                        dates[1]= "April"
                    elif dates[1] in ["May","MAY"]:
                        dates[1]= "May"
                    elif dates[1] in ["JUN","Jun","June"]:
                        dates[1]= "June"
                    elif dates[1] in ["JULY","July","JUL"]:
                        dates[1]= "July"
                    if dates[0]== "fifteenth" :
                        dates[0]="15"
                    elif (dates[0]=="sixteenth") |(dates[0]=="This 16 "):
                        dates[0]="16"
                    elif dates[0]=="Twenty":
                        dates[0]="20" 
                    elif dates[0]=="Twentyseventh":
                        dates[0]="27" 
                    elif (dates[0]=="Twenty Eighth") |(dates[0]=="TwentyEight") |(dates[0]=="TwentyEight ") |(dates[0]=="Twentyeight"):
                        dates[0]="28"
                    elif (dates[0]=="Ninth") |(dates[0]=="Ninth ")|(dates[0]=="Augusta 9th ") |(dates[0]=="ninth") |(dates[0]=="ninth ")|(dates[0]=="Augusta 9th"):
                        dates[0]="9"
                    
                    elif dates[0]=='Seventh':
                        dates[0]="7" 
                    elif (dates[0]=='The') |(dates[0]=="The 9th") :
                        dates[0]="9"
                    elif dates[0]=="the 9" :
                        dates[0]="9"
                    elif (dates[0]=="Eighth ") |(dates[0]=="Eighth") |(dates[0]=="eighth") |(dates[0]=="eighth "):
                        dates[0]="8"
                    elif dates[0]=="Seventh ":
                        dates[0]="7"
                    elif (dates[0]=="Twenty ninth ")| (dates[0]=="Twenty ninth") | (dates[0]=="twenty ninth ")|(dates[0]=="Twenty Ninth ")|(dates[0]=="Twenty Ninth") :
                        dates[0]="29"
                    elif (dates[0]=="Thirty") |(dates[0]=="Thirty ") | (dates[0]=="thirtieth "):
                        dates[0]="30"
                    elif (dates[0]=="Thirty first ") |(dates[0]=="Thirty first"):
                        dates[0]="31"
                  
                    
                    elif dates[0]=="first ":
                        dates[0]="1"
                    elif (dates[0]=="Second ") |(dates[0]=="Second") |(dates[0]=="second")|(dates[0]=="second "):
                        dates[0]="2"
                    elif dates[0]=="fourth ":
                        dates[0]="4"
                    elif dates[0]=="tenth ":
                        dates[0]="10"
                    elif (dates[0]=="Fifth ")|(dates[0]=="Fifth") |(dates[0]=="fifth ")|(dates[0]=="fifth"):
                        dates[0]="5"
                    elif (dates[0]=="This") | (dates[0]=="This 7th ")|(dates[0]=="seventh ") |(dates[0]=="seventh") |(dates[0]=="Seventh "):
                        dates[0]="7"
                    elif (dates[0]=="This 31st") |(dates[0]=="Thirty-first "):
                        dates[0]="31"
                    elif (dates[0]=="Thirtieth") |(dates[0]=="Thirtieth ") |(dates[0]=="Thirty"):
                        dates[0]="30"
                    elif dates[0]=="twenty fifth ":
                        dates[0]="25"
                    elif dates[0]=="Twenty seventh ":
                        dates[0]="27"
                    elif (dates[0]=="Twenty sixth ") |(dates[0]=="Twentysixth ")|(dates[0]=="Twentysixth") |(dates[0]=="twentysixth "):
                        dates[0]="26"
                    elif dates[0]=="third ":
                        dates[0]="3" 
                    elif dates[0]=="third":
                        dates[0]="3"
                    elif dates[0]=="Third ":
                        dates[0]="3"
                    elif dates[0]=="Third":
                        dates[0]="3"
                    
                    elif dates[0]=="Tenth":
                        dates[0]="10" 
                    elif dates[0]=="Augusta":
                        dates[0]="9" 
                    elif (dates[0]=="this_sixth ")|(dates[0]=="sixth ")| (dates[0]=="Sixth") | (dates[0]=="Sixth ") |(dates[0]=="this sixth "):
                        dates[0]="6" 
                        
                    elif dates[0]=="Tenth ":
                        dates[0]="10"
                    elif (dates[0]=="twentieth")|(dates[0]=="This 20th "):
                        dates[0]="20"
                    elif (dates[0]=="twenty fourth ") |(dates[0]=="twenty fourth") |(dates[0]=="Twenty fourth ") |(dates[0]=="Twenty fourth"):
                        dates[0]="24"
                    elif (dates[0]=="fifteenth ") |(dates[0]=="Fifteenth ") |(dates[0]=="Fifteenth"):
                        dates[0]="15"
                    elif dates[0]=="twentieth ":
                        dates[0]="20"
                    elif dates[0]=="Eighteenth ":
                        dates[0]="18"
                    elif dates[0]=="Eighteenth":
                        dates[0]="18"
                    elif (dates[0]=="Nineteenth ") |(dates[0]=="Nineteenth"):
                        dates[0]="19"
                    elif dates[0]=="First ":
                        dates[0]="1"
                    elif dates[0]=="First":
                        dates[0]="1"
                    elif (dates[0]=="Sixteenth ")|(dates[0]=="this 16") |(dates[0]=="Sixteenth"):
                        dates[0]="16"
                   
                    elif dates[0]=="twenty third ":
                        dates[0]="23"
                    elif dates[0]=="twenty third":
                        dates[0]="23"
                    elif dates[0]=="Twenty third ":
                        dates[0]="23"
                    elif dates[0]=="Twenty Second ":
                        dates[0]="22"
                    elif dates[0]=="Twenty Second":
                        dates[0]="22"
                    elif dates[0]=="Twenty Eighth ":
                        dates[0]="28"
                    elif dates[0]=="Twentysenventh ":
                        dates[0]="27"
                    elif dates[0]=="twenty third ":
                        dates[0]="23"
                    elif (dates[0]=="twenty third") |(dates[0]=="Twentythird")|(dates[0]=="Twentythird "):
                        dates[0]="23"
                    elif (dates[0]=="fourteenth ") | (dates[0]=="fourteenth") | (dates[0]=="Fourteenth") | (dates[0]=="Fourteenth "):
                        dates[0]="14"
                    elif (dates[0]=="Eleventh ") | (dates[0]=="Eleventh") |(dates[0]=="eleventh ") :
                        dates[0]="11"
                    elif (dates[0]=="Thirteenth") | (dates[0]=="Thirteenth ") :
                        dates[0]="13"
                    elif (dates[0]=="seventeenth ") |(dates[0]=="seventeenth") |(dates[0]=="Seventeenth ") :
                        dates[0]="17"
                     
                     
                     
                    
                        
                    j = dates[0]+"-"+dates[1]+"-"+dates[2]
                    ng.append(j)
            
            
    NG.append(ng)   

In [26]:
order_onto["standard_date-ontonotes"]=NG

In [27]:
order_onto.to_csv("order_onto.csv",index=False)

# Ner english large 

In [28]:
lst=[]
label=[]
Per=[]
ORG=[]
LOC=[]
Other_name=[]
from flair.data import Sentence

order_large=pd.DataFrame({"order_text":"", "PERSON-nerenglarge":[], "LOC-nerenglarge":"","ORG-nerenglarge":""})
for i in range(7668):
        sentencess =Sentence(order_text[i])
        order_large.loc[i,"order_text"]=order_text[i]
        tagger2.predict(sentencess)
        dc={}
        P=[]
        L=[]
        G=[]
        O=[]
        Other=[]
        for entity in sentencess.get_spans('ner'):
                     
                     if (entity.get_label("ner").score > 0.90) & (entity.get_label("ner").value != "CARDINAL") & (entity.get_label("ner").value != "LAW")& (entity.get_label("ner").value != "QUANTITY") & (entity.get_label("ner").value != "WORK_OF_ART") & (entity.get_label("ner").value != "LAW")& (entity.get_label("ner").value != "QUANTITY") & (entity.get_label("ner").value != "EVENT")  :
                        dc[entity.text]= entity.get_label("ner").value 
        
        
        for j in dc :
                if dc[j]=="PER":
                      P.append(j)
                elif (dc[j]=="ORG") :
                      O.append(j)
                elif  (dc[j]=="MISC") :
                      Other.append(j)
                elif dc[j]=="LOC":
                      L.append(j)
               
        Per.append(P)
        ORG.append(O)
        LOC.append(L)
        Other_name.append(Other)
            

order_large["PERSON-nerenglarge"]=Per
order_large["LOC-nerenglarge"]=LOC
order_large["ORG-nerenglarge"]=ORG    
order_large["other_namelarge"] =Other_name 

2022-06-27 19:27:13,926 Warning: An empty Sentence was created! Are there empty strings in your dataset?
2022-06-27 19:28:03,766 Warning: An empty Sentence was created! Are there empty strings in your dataset?
2022-06-27 19:34:15,305 Warning: An empty Sentence was created! Are there empty strings in your dataset?


In [29]:
order_large.head()

,order_text,PERSON-nerenglarge,LOC-nerenglarge,ORG-nerenglarge,other_namelarge
0,R G. R.R . THIS ORDER NOT TRANSFERABLE . U.S. ...,"[Cranford, Lewis B. Parsons, Gaston R.R]","[U.S., Raleigh N.C, Warrenton N.C, Raleigh N.C...","[Samuel Gale Co, Regiment Freedman, Rail River...",[]
1,"Certified Copy of Order , Extract , within tra...","[Robinson Clinton, E. Crawford]",[Raleigh N.C],[],[]
2,"Bureau of Refugees , Freedmen Abandoned Lands ...","[KELLEY, EDWARD McPHERSON, E. WHITTLESEY, JONA...","[State of North Carolina, Raleigh, N.C., Alaba...","[Bureau of Refugees , Freedmen Abandoned Lands...",[]
3,"HEADQUARTERS DEPARTMENT OF NORTH CAROLINA , Ra...",[Joseph E. Johnston],"[NORTH CAROLINA, Raleigh, North Carolina, U.S....",[Government],[]
4,IV . Animals sold counties collected unless wo...,"[J A . Campbell, James H Anderson]","[U.S., U.S. C.S, C.S]",[Government],[]


In [30]:
order_large.to_csv("order_large.csv",index=False)

# Ner english

In [31]:
lst=[]
label=[]
Per=[]
ORG=[]
LOC=[]
Other_name=[]
from flair.data import Sentence

order_ner=pd.DataFrame({"order_text":"", "PERSON-nerenglish":"","LOC-nerenglish":"","ORG-nerenglish":[]})
for i in range(20):
        sentencess =Sentence(order_text[i])
        order_ner.loc[i,"order_text"]=order_text[i]
        tagger1.predict(sentencess)
        dc={}
        P=[]
        L=[]
        O=[]
        Other=[]
        for entity in sentencess.get_spans('ner'):
                     
                     if (entity.get_label("ner").score > 0.90) & (entity.get_label("ner").value != "CARDINAL") & (entity.get_label("ner").value != "LAW")& (entity.get_label("ner").value != "QUANTITY") & (entity.get_label("ner").value != "WORK_OF_ART") & (entity.get_label("ner").value != "LAW")& (entity.get_label("ner").value != "QUANTITY") & (entity.get_label("ner").value != "EVENT")  :
                        dc[entity.text]= entity.get_label("ner").value 
        
        
        for j in dc :
                if dc[j]=="PER":
                      P.append(j)
                elif (dc[j]=="ORG")  :
                      O.append(j)
                elif  (dc[j]=="MISC") :
                      Other.append(j)
                elif dc[j]=="LOC":
                      L.append(j)
               
        Per.append(P)
        ORG.append(O)
        LOC.append(L)
        Other_name.append(Other)
            

order_ner["PERSON-nerenglish"]=Per
order_ner["LOC-nerenglish"]=LOC
order_ner["ORG-nerenglish"]=ORG 
order_ner["Other_name-nerenglish"]=Other_name 

In [32]:
order_ner.head()

,order_text,PERSON-nerenglish,LOC-nerenglish,ORG-nerenglish,Other_name-nerenglish
0,R G. R.R . THIS ORDER NOT TRANSFERABLE . U.S. ...,[],"[Ral, Gaston R.R, Raleigh N.C., Broadway, New ...",[Cranford],[]
1,"Certified Copy of Order , Extract , within tra...",[],[],[],[]
2,"Bureau of Refugees , Freedmen Abandoned Lands ...",[JONATHAN WORTH],"[Raleigh, N.C., United States, Alabama, North ...","[Officers Agents of Bureau, House of Represent...",[]
3,"HEADQUARTERS DEPARTMENT OF NORTH CAROLINA , Ra...",[Joseph E. Johnston],"[NORTH CAROLINA, Raleigh, North Carolina]",[],[]
4,IV . Animals sold counties collected unless wo...,[James H Anderson],[U.S.],[],[C.S]


In [33]:
order_ner.to_csv("order_ner.csv",index=False)

In [34]:
order_onto["PERSON-nerenglarge"]=order_large["PERSON-nerenglarge"]
order_onto["LOC-nerenglarge"]=order_large["LOC-nerenglarge"]
order_onto["ORG-nerenglarge"]=order_large["ORG-nerenglarge"]
order_onto["other_namelarge"]=order_large["other_namelarge"]
order_onto["PERSON-nerenglish"]=order_ner["PERSON-nerenglish"]
order_onto["LOC-nerenglish"]=order_ner["LOC-nerenglish"]
order_onto["ORG-nerenglish"]=order_ner["ORG-nerenglish"]
order_onto["Other_name-nerenglish"]=order_ner["Other_name-nerenglish"]

In [35]:
to_compare_result_order=order_onto

In [36]:
to_compare_result_order.to_csv("to_compare_result_order.csv", index=False)